# Introduction to bokeh

## Bokeh server app
---

<img src='./images/logos.3.600.wide.png' height='250' width='300' style="float:right">

We briefly mentioned one of the major donwsides to the client-side version of Bokeh: limitations within the browser in terms of how much data can be handled at any given time.

Bokeh Server attempts to get past that and enable the browser and the data to be synced more effectively:
 

* **Event response**: respond to User Interface (UI) and tool events generated in the browser with computations or queries using the full power of Python
* **Server-side updates**: automatically push server-side updates to the UI (i.e. widgets or plots in a browser)
* **Streaming updates** use periodic, timeout, and asynchronous callbacks to drive streaming updates

Bokeh Server can be used in several modes:

* local / individual use
* deployable application

Our focus will be on local and individual use. This should get you pointed in the right direction should you want to create a deployable application.

# Behind the scenes
---

* Behind the scenes, Bokeh Server will create a document specific to each browser session.
* Each document allows users to access and manipulate the data.<br>


<img src='./images/bokeh_serve.svg' width='700' style="float:center">

# Bokeh Server stock example

We will stand up a Bokeh Server that produces visualizations focused on stock exchange data

* Prep your environment
* Download the app and data
* Generate the code
* Run the server


## Prep  your environment


### Linux/Mac
```bash
mkdir bserver
cd bserver
conda create -n bserver python=3
source activate bserver
conda install bokeh jupyter ipython pandas
```

### WIndows
```bash
mkdir bserver
cd bserver
conda create -n bserver python=3
activate bserver
conda install bokeh jupyter ipython pandas
```

## Download the app and data, part 1

1. Navigate to the Bokeh github site: https://github.com/bokeh/bokeh 
1. Click on the **Clone or Downland** button
1. Click on the **Download Zip** button <img src='./images/github_download.png' height='1000' width='600'><br>
1. Save the file to your `bserver` folder
1. Unzip the file, if necessary


## Download the app and data, part 2

1. Type the following on the command line to download the datafiles for the **STOCKS** application. NOTE: this presumes that you are **already** in your bserver folder:

```bash
cd bokeh-master/examples/app/stocks/
python download_sample_data.py
```
1. Back out of the stocks folder, using the following command:

```bash
cd ..
```


## Generate the code

In this case, we will be reviewing the script that is already present in the `stocks` folder.

* **Notebook version below is NOT executable:** The next few cells are edited with Markdown, so they are **NOT** executable. 

* **Direct access:** We will simply examine the code here in the notebook, but to access it directly OR to execute the code, we will use the `main.py` file from the `stocks`. 

* **Edited for clarity:** the version in the notebook is edited for clarity - we strip out duplicate OR similar lines and we potentially cut back on details covered elsewhere in the workshop, to focus on items pertinent to using and interacting with the Bokeh Server. 

* **`main.py`:** If you want to edit the code, you may do so by opening `main.py` with your favorite text editor.





### HEADER --------------------------------------------------
```python
''' Create a simple stocks correlation dashboard.

Choose stocks to compare in the drop down widgets, and make selections
on the plots to update the summary and histograms accordingly.

<snip>

When the server is running navigate to the URL:

    http://localhost:5006/stocks

.. _README: https://github.com/bokeh/bokeh/blob/master/examples/app/stocks/README.md

'''
```


### IMPORTS -------------------------------------------------

```python
<snip>

from bokeh.io import curdoc
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import PreText, Select
from bokeh.plotting import figure

```


### CREATE OR LOAD DATA/ENRICHMENTS -------------------------

```python

# Combine the name of the directory with the subdirectory 'daily'
# to provide access to all the data files

DATA_DIR = join(dirname(__file__), 'daily')

# Assign a set of default stocks to display in the Ticker dropdowns

DEFAULT_TICKERS = ['AAPL', 'GOOG', 'INTC', 'BRCM', 'YHOO']

def nix(val, lst):
    '''Returns a list of elements that does not contain the provided value
    
    i.e. can be used to keep two dropdown menus synced so that the item selected
    in one menu does not show up in the other menu
    '''
    
    return [x for x in lst if x != val]

@lru_cache()
def load_ticker(ticker):
    '''Given a ticker symbol (i.e. 'GOOG') return a DataFrame with the
    date as an index, and the closing price and difference in price from
    one day to the previous day.
    '''
    
    fname = join(DATA_DIR, 'table_%s.csv' % ticker.lower())
    data = pd.read_csv(fname, header=None, parse_dates=['date'],
                       names=['date', 'foo', 'o', 'h', 'l', 'c', 'v'])
    data = data.set_index('date')
    return pd.DataFrame({ticker: data.c, ticker+'_returns': data.c.diff()})

@lru_cache()
def get_data(t1, t2):
    '''Given two ticker symbols, uses load_ticker() to load the data & returns a DataFrame:
    NOTE: 
    * the two data sets are concatenated
    * NA values are dropped
    * the columns are renamed
    '''
    
    df1 = load_ticker(t1)
    df2 = load_ticker(t2)
    data = pd.concat([df1, df2], axis=1)
    data = data.dropna()
    data['t1'] = data[t1]
    data['t2'] = data[t2]
    data['t1_returns'] = data[t1+'_returns']
    data['t2_returns'] = data[t2+'_returns']
    return data

# set up widgets

# PreText() creates a widget for displaying a block of pre-formatted text in an HTML <pre> tag
stats = PreText(text='', width=500)

# Select() creates a dropdown widget for selecting a single value.
# In this case, the default, initial value for each dropdown is 
# set and the remaining values are generated by the nix() function
ticker1 = Select(value='AAPL', options=nix('GOOG', DEFAULT_TICKERS))
ticker2 = Select(value='GOOG', options=nix('AAPL', DEFAULT_TICKERS))

# set up plots

# ColumnDataSource() maps the names of columns to sequences or arrays, often using a dictionary OR DataFrame
# This allows you to then reference the data by column name
# as well as access various functions to allow the manipulation
# of that data.
source = ColumnDataSource(data=dict(date=[], t1=[], t2=[], t1_returns=[], t2_returns=[]))
source_static = ColumnDataSource(data=dict(date=[], t1=[], t2=[], t1_returns=[], t2_returns=[]))

# tools defines a set of tools to display
tools = 'pan,wheel_zoom,xbox_select,reset'

```

## GENERATE:

### FIGURE -----------------------------------------

```python

# Produces a figure that will end up hosting a circle() plot
corr = figure(plot_width=350, plot_height=350, 
              tools='pan,wheel_zoom,box_select,reset')            
```

### GLYPHS -----------------------------------------
```python

# This circle() plot correlates the returns of Stock 1 and Stock 2.
# Notice the nonselection vs selection alphas.
# Points that are selected vs not selected using a selection tool 
# can have different levels of transparency (i.e. they can be greyed
# out)
corr.circle('t1_returns', 't2_returns', size=2, source=source,
            selection_color="orange", alpha=0.6, 
            nonselection_alpha=0.1,
            selection_alpha=0.4)
```

### FIGURE -----------------------------------------
```python

# Note, this refers to the tools variable we defined in the previous Jupyter cell
# That tools variable mentioned xbox_select. It is also mentioned here in 
# regards to the active_drag argument.
# This type of box_select tool selects, by default, across the x-axis.
ts1 = figure(plot_width=900, plot_height=200, 
             tools=tools,
             x_axis_type='datetime',
             active_drag="xbox_select")
```

### GLYPHS -----------------------------------------
```python
ts1.line('date', 't1', source=source_static)
ts1.circle('date', 't1', size=1, source=source, color=None, selection_color="orange")
```


### ADD ATTRIBUTES, ANNOTATIONS, INTERACTIONS ---------------

```python
# set up callbacks

def ticker1_change(attrname, old, new):
    '''Updates the options to be displayed in ticker 2.
    '''
    
    ticker2.options = nix(new, DEFAULT_TICKERS)
    update()


def update(selected=None):
    '''Updates all the data associated with the display by 
    reading in data from multiple sources.
    '''
    
    t1, t2 = ticker1.value, ticker2.value

    data = get_data(t1, t2)
    source.data = source.from_df(data[['t1', 't2', 't1_returns', 't2_returns']])
    source_static.data = source.data

    update_stats(data, t1, t2)

    corr.title.text = '%s returns vs. %s returns' % (t1, t2)
    ts1.title.text, ts2.title.text = t1, t2

def update_stats(data, t1, t2):
    '''Updates the data to be displayed by the stats object, given a dataframe
    and two ticker symbols
    '''
    
    stats.text = str(data[[t1, t2, t1+'_returns', t2+'_returns']].describe())
```


### ADD ATTRIBUTES, ANNOTATIONS, INTERACTIONS (part 2) ------

```python
# ticker1 was instantiated using the Select() function.
# When the attribute 'value' changes, the Python function ticker1_change()
# is called as a Callback()

ticker1.on_change('value', ticker1_change)
ticker2.on_change('value', ticker2_change)

def selection_change(attrname, old, new):
    '''Update parameters when the selection changes.
    '''
    
    t1, t2 = ticker1.value, ticker2.value
    data = get_data(t1, t2)
    selected = source.selected['1d']['indices']
    if selected:
        data = data.iloc[selected, :]
    update_stats(data, t1, t2)

# source was instantiated using the ColumnDataSource() function.
# When the attribute 'selected' changes, the Python function selection_change()
# is called as a Callback()

source.on_change('selected', selection_change)
```


### ADD ATTRIBUTES, ANNOTATIONS, INTERACTIONS (part 3) ------

```python
# Set up layout
#   The layout is predominantly composed of column() and row() objects
#   Each column() and row() object may contain other columns or rows
#   or actual glyph objects.
#   Elements in a column() object are displayed in a column
#   Elements in a row() object are displayed in a row


widgets = column(ticker1, ticker2, stats)
main_row = row(corr, widgets)
series = column(ts1, ts2)
layout = column(main_row, series)

```

<img src='./images/layouts.png' height='1000' width='600'>

### CREATE OUTPUTS ------------------------------------------

```python
# initialize

update()

curdoc().add_root(layout)
curdoc().title = "Stocks"

```

## Run the server
---

From **within** the **/stocks** folder, execute the following command from the command line:

1. `bokeh serve --show .`

## Navigation
---

| Previous | Up | Next |
|:----|:-----:|-----:|
| <<< [What Went Wrong](./what_went_wrong.ipynb) | [Table of Contents](./README.md) | [Charting Your Course](./charting_your_course.ipynb) >>> |